In [1]:
import chess_gameplay as chg

In [2]:
# Initialize a pair of agents. These agents are initialized with no model to make judgements
# about the move options presented to them, so will randomly select from their set of legal moves.

agents = [chg.Agent(), chg.Agent()]

In [3]:
# Now we can make our random agents play a game against each other. They're not very good and they don'try
# actually even know what checkmate is, so we're pretty unlikely to get anything other than a draw.

game_result = chg.play_game(
    table = 1,                                              # Used to send games to one tournament table or another
    agents = {'white': agents[0], 'black': agents[1]},      # We specify which agent is playing with which pieces
    max_moves = 5,                                          # Play until each agent has made up to 10 moves
    min_seconds_per_move = 0.1,                             # Artificially slow the game down for better watching
    verbose = True,                                         # Tell us what moves each agent makes and the outcome
    poseval = True                                          # Use stockfish to evaluate each position and fancy plot
)

white_points, black_points = game_result["white"]["points"], game_result["black"]["points"]
print(f"White: {white_points}, Black: {black_points}")

white: b3
black: e5
white: g3
black: f5
white: b4
black: b6
white: Na3
black: g5
white: d3
black: Na6
Draw or timeout after 5 moves.
White: 1.0, Black: -1.0


In [5]:
# We can also play a "tournament" which is a series of games in which a pair of agents take turns in playing
# as black and then as white. Ensuring each agent gets a turn at white before deciding the winner removes the
# inherent bias / advantage of playing as white. On gameday we use stockfish to evaluate draw positions and 
# decide which player has the better position. If agents draw at the end of each two games then another two
# games are played and so on until eventually one agent wins decisively.

tournament_results, tournament_game_results = chg.play_tournament(
    table = 1,
    agents = agents,
    max_games = 4,
    max_moves = 2,
    min_seconds_per_move = 0.1,
    verbose=False,
    poseval = True
)

agent0_points, agent1_points = tournament_results["agent0"], tournament_results["agent1"]
print(f"Agent0: {agent0_points}, Black: {agent1_points}")

DRAW!
Agent0: 2.0, Black: -2.0


In [9]:
# Now let's check back in with our training model! First load in the trained model weights

import torch
import yaml
from model import Model

model_config = yaml.safe_load(open("model_config.yaml"))
model = Model(**model_config)

checkpoint = torch.load("/root/outputs/chess-eval/<experiment_id>/checkpoint.pt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["model"])

/root/.chess/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


<All keys matched successfully>

In [11]:
# Now we can inject our model into one of our agents, and play a game! (hopefully our model-powered agent wins!)

agents = [chg.Agent(model), chg.Agent()]

game_result = chg.play_game(
    table = 1,                                              # Used to send games to one tournament table or another
    agents = {'white': agents[0], 'black': agents[1]},      # We specify which agent is playing with which pieces
    max_moves = 5,                                          # Play until each agent has made up to 10 moves
    min_seconds_per_move = 0.1,                             # Artificially slow the game down for better watching
    verbose = True,                                         # Tell us what moves each agent makes and the outcome
    poseval = True                                          # Use stockfish to evaluate each position and fancy plot
)

/root/demo-chess/chess_gameplay.py:78: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  scores = self.model(torch.tensor(options))


white: c3
black: g6
white: Nf3
black: a6
white: h4
black: b5
white: d3
black: Ra7
white: d4
black: Ra8
Draw or timeout after 5 moves.


In [ ]:
# And also play them in a fair tournament

tournament_results, tournament_game_results = chg.play_tournament(
    table = 1,
    agents = agents,
    max_games = 4,
    max_moves = 2,
    min_seconds_per_move = 0.1,
    verbose=False,
    poseval = True
)

agent0_points, agent1_points = tournament_results["agent0"], tournament_results["agent1"]
print(f"Agent0: {agent0_points}, Black: {agent1_points}")